# LSA Topic Modelling

## Read Data

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Data didapatkan dari proses scrapping data youtube


proses scrapping dapat dilihat pada halaman berikut [scrapping_comment_yt](https://hamedayani114.github.io/prosaindata/scrapping_comment_yt.html)

In [ ]:
import pandas as pd
df=pd.read_csv('input teks/datacomment_cnn_fix.csv')
df

,comment,comment (clean)
0,ID\nSkip navigation\nSign in\nWorship Piano: B...,id skip navigation sign ini worship piano begi...
1,"Pak Prabowo Subianto saya mohon,\nAnda jangan ...",pak prabowo subianto saya mohon anda jangan be...
2,gw dan sekeluarga adalah loyalis PDI Perjuanga...,gue dan sekeluarga adalah loyalis pdi perjuang...
3,"ganjar salah satu yang menolak israel, berakib...",ganjar salah satu yang menolak israel berakiba...
4,"Pak Prabowo sekarang auranya adem,beda waktu n...",pak prabowo sekarang auranya adem beda waktu n...
...,...,...
1160,Di,di
1161,Siapa pula yg mau pilih prabowo....history mas...,siapa pula yang mau pilih prabowo history masa...
1162,Lembaga survei bayaran\nAslinya pak Anies yg t...,lembaga survei bayaran aslinya pak anies yang ...
1163,Siapapun calonya presiden 2024\nKami rakyat t...,siapapun calonya presiden kami rakyat tetap an...


## Modelling

In [ ]:
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
# import nltk stopwords indo
from nltk.corpus import stopwords
stopwords = stopwords.words('indonesian')

# Membentuk matriks dokumen x kata
tokenizer = RegexpTokenizer(r'\w+')
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words=stopwords,
                        tokenizer = tokenizer.tokenize)

tfidf_matrix = vectorizer.fit_transform(df['comment (clean)'])

# Melakukan dekomposisi matriks dengan SVD
svd_model = TruncatedSVD(n_components=4)
lsa_matrix = svd_model.fit_transform(tfidf_matrix)

### bobot kata terhadap masing masing topik

In [ ]:
# bobot kata terhadap masing masing topik
terms = vectorizer.get_feature_names_out()

for index, component in enumerate(svd_model.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:2]
    print("Topic "+str(index)+": ",top_terms_key)

Topic 0:  [('prabowo', 0.7521357956218078), ('ganjar', 0.33720181031628493)]
Topic 1:  [('ganjar', 0.7502805139651377), ('pilih', 0.18626380182167412)]
Topic 2:  [('mahfud', 0.5859716215798411), ('md', 0.5193567117815464)]
Topic 3:  [('anis', 0.5762952341786094), ('pilih', 0.38496202207521185)]


### bobot setiap topik terhadap dokumen

In [ ]:
# bobot setiap topik terhadap  dokumen
df_lsa = pd.DataFrame(lsa_matrix, columns=["Topik 0", "Topik 1", "Topik 2", "Topik 3"])
df_lsa = pd.concat([df["comment (clean)"], df_lsa], axis=1)
df_lsa['Topik']= df_lsa[['Topik 0', 'Topik 1', 'Topik 2', 'Topik 3']].apply(lambda x: x.argmax(), axis=1)

df_lsa

,comment (clean),Topik 0,Topik 1,Topik 2,Topik 3,Topik
0,id skip navigation sign ini worship piano begi...,0.152407,0.051588,-0.010255,0.048219,0
1,pak prabowo subianto saya mohon anda jangan be...,0.307148,0.188855,0.026351,-0.118356,0
2,gue dan sekeluarga adalah loyalis pdi perjuang...,0.054244,0.070545,-0.004968,0.014251,1
3,ganjar salah satu yang menolak israel berakiba...,0.071627,0.146227,0.047391,-0.040094,1
4,pak prabowo sekarang auranya adem beda waktu n...,0.098525,-0.021477,-0.032593,-0.031965,0
...,...,...,...,...,...,...
1160,di,0.000000,0.000000,0.000000,0.000000,0
1161,siapa pula yang mau pilih prabowo history masa...,0.107983,0.008258,-0.050534,0.058642,0
1162,lembaga survei bayaran aslinya pak anies yang ...,0.030155,0.034998,-0.035307,0.110391,3
1163,siapapun calonya presiden kami rakyat tetap an...,0.118906,0.049716,-0.129403,0.273918,3


In [ ]:
df_lsa['Topik'].value_counts()

0    567
3    317
1    221
2     60
Name: Topik, dtype: int64